# Estimar a base completa

In [1]:
#Imports
import pandas as pd     #Manipulação dos dados
import numpy as np      #Tarefas matemáticas
##Machine learning
from pycaret.regression import * 
#Definir o formato de exibição tipo float para evitar notação científica
pd.options.display.float_format = '{:.2f}'.format
##Eliminar os warnings
import warnings
warnings.filterwarnings("ignore")
##Ver todas as colunas do data frame
pd.set_option('display.max_columns', None)

## Visão geral dos dados

In [2]:
df = pd.read_csv('base_dados.csv',sep = ';')
#Visualização
df.head(3)

,ID_RESIDENCIA,PRECO,AREA,QUARTOS,BANHEIROS,ANDARES,FLAG_CENTRO,FLAG_QUARTO_HOSPEDE,FLAG_PORAO,FLAG_AGUA_MORNA,FLAG_AR_CONDICIONADO,VAGAS_ESTACIONAMENTO,FLAG_AREA_PREFERENCIAL,MOBILIADA
0,A1,13300000,7420,4.00,2.00,3,SIM,NAO,NAO,NaN,SIM,2.00,SIM,SIM
1,A2,12250000,8960,4.00,4.00,4,SIM,NAO,NaN,NAO,SIM,3.00,NAO,SIM
2,A3,12250000,9960,NaN,2.00,2,SIM,NAO,SIM,NAO,NAO,2.00,SIM,PARCIALMENTE


## Carregar o experimento criado

In [4]:
model_lasso = load_model('lasso_model')
#Pipeline do modelo criado
model_lasso

Transformation Pipeline and Model Successfully Loaded


Pipeline(memory=Memory(location=None),
         steps=[('numerical_imputer',
                 TransformerWrapper(include=['AREA', 'QUARTOS', 'BANHEIROS',
                                             'ANDARES',
                                             'VAGAS_ESTACIONAMENTO'],
                                    transformer=SimpleImputer(strategy='median'))),
                ('categorical_imputer',
                 TransformerWrapper(include=['FLAG_CENTRO',
                                             'FLAG_QUARTO_HOSPEDE',
                                             'FLAG_PORAO', 'FLAG_AGUA_MORNA',
                                             'FLAG_AR_CONDICIONADO',
                                             'FLAG...
                                                                         'data_type': dtype('O'),
                                                                         'mapping': NAO             0
PARCIALMENTE    1
SIM             2
NaN            -1
dtype: int64}]))),
                ('onehot_encoding',
                 TransformerWrapper(include=['MOBILIADA'],
                                    transformer=OneHotEncoder(cols=['MOBILIADA'],
                                                              handle_missing='return_nan',
                                                              use_cat_names=True))),
                ('normalize', TransformerWrapper(transformer=MinMaxScaler())),
                ('actual_estimator', Lasso(alpha=1000, random_state=1935))])

## Estimação

In [5]:
estimativa = predict_model(model_lasso, data = df)
#Visualização
estimativa.head(3)

,ID_RESIDENCIA,AREA,QUARTOS,BANHEIROS,ANDARES,FLAG_CENTRO,FLAG_QUARTO_HOSPEDE,FLAG_PORAO,FLAG_AGUA_MORNA,FLAG_AR_CONDICIONADO,VAGAS_ESTACIONAMENTO,FLAG_AREA_PREFERENCIAL,MOBILIADA,PRECO,prediction_label
0,A1,7420,4.00,2.00,3,SIM,NAO,NAO,NaN,SIM,2.00,SIM,SIM,13300000,8260934.10
1,A2,8960,4.00,4.00,4,SIM,NAO,NaN,NAO,SIM,3.00,NAO,SIM,12250000,10509614.39
2,A3,9960,NaN,2.00,2,SIM,NAO,SIM,NAO,NAO,2.00,SIM,PARCIALMENTE,12250000,7657495.96


### Reestruturar a coluna das estimativas

In [6]:
estimativa['ESTIMATIVA_PRECO'] = estimativa['prediction_label']

### Filtrar as variáveis importantes

In [7]:
df_final = estimativa[['ID_RESIDENCIA', 'PRECO', 'ESTIMATIVA_PRECO']]
#Visualização
df_final.head(3)

,ID_RESIDENCIA,PRECO,ESTIMATIVA_PRECO
0,A1,13300000,8260934.10
1,A2,12250000,10509614.39
2,A3,12250000,7657495.96


### Calcular o MAPE

In [8]:
from sktime.performance_metrics.forecasting import mean_absolute_percentage_error

In [10]:
100 * mean_absolute_percentage_error(df_final.PRECO, df_final.ESTIMATIVA_PRECO) #17%

17.351653876603155

### Salvar em CSV a base estimada

In [11]:
#Salvar a base
df_final.to_csv('df_base_completa_estimada.csv', header=True, index=False, sep=';')